## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-15-14-51-21 +0000,bbc,Netanyahu does not rule out further strikes on...,https://www.bbc.com/news/articles/czxw4xkxg51o...
1,2025-09-15-14-50-25 +0000,nypost,Community honors life of 10-year-old Harper Mo...,https://nypost.com/2025/09/15/us-news/communit...
2,2025-09-15-14-50-00 +0000,nyt,The Long-Term Unemployed Today? College Grads.,https://www.nytimes.com/2025/09/15/business/lo...
3,2025-09-15-14-44-11 +0000,nyt,F.B.I. Head Says Note and D.N.A. Link Suspect ...,https://www.nytimes.com/2025/09/15/us/charlie-...
4,2025-09-15-14-43-00 +0000,wsj,Rubio Seeks to Move Beyond Israeli Strike in Q...,https://www.wsj.com/world/middle-east/rubio-se...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2308/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
112,trump,34
26,kirk,23
90,charlie,17
81,china,13
60,new,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
236,2025-09-14-15-50-46 +0000,nypost,Trump still furious over Charlie Kirk’s murder...,https://nypost.com/2025/09/14/us-news/utah-gov...,101
178,2025-09-14-23-51-33 +0000,nypost,Trump blasts ‘the left’ and announces ‘major i...,https://nypost.com/2025/09/14/us-news/trump-bl...,92
211,2025-09-14-19-31-04 +0000,nypost,Eric Trump: Assassination of Charlie Kirk has ...,https://nypost.com/2025/09/14/us-news/eric-tru...,88
36,2025-09-15-13-44-49 +0000,nypost,Chilling new video shows Charlie Kirk suspect ...,https://nypost.com/2025/09/15/us-news/video-sh...,87
162,2025-09-15-02-00-00 +0000,wsj,A bullet casing found after the killing of Cha...,https://www.wsj.com/politics/policy/fascist-en...,73


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
236,101,2025-09-14-15-50-46 +0000,nypost,Trump still furious over Charlie Kirk’s murder...,https://nypost.com/2025/09/14/us-news/utah-gov...
36,47,2025-09-15-13-44-49 +0000,nypost,Chilling new video shows Charlie Kirk suspect ...,https://nypost.com/2025/09/15/us-news/video-sh...
27,39,2025-09-15-14-03-11 +0000,nypost,"Bessent says US, China have reached ‘framework...",https://nypost.com/2025/09/15/business/us-chin...
34,36,2025-09-15-13-52-39 +0000,wapo,"Rubio downplays U.S. tensions with Israel, pla...",https://www.washingtonpost.com/national-securi...
226,32,2025-09-14-17-33-46 +0000,nyt,White House Seeks More Supreme Court Security ...,https://www.nytimes.com/2025/09/14/us/politics...
86,31,2025-09-15-11-00-00 +0000,missionlocal,S.F. will be one of the smallest losers in Tru...,https://missionlocal.org/2025/09/sf-smallest-l...
18,30,2025-09-15-14-25-00 +0000,wsj,Tesla Shares Rise on Elon Musk’s $1 Billion St...,https://www.wsj.com/tech/tesla-stock-elon-musk...
15,29,2025-09-15-14-27-00 +0000,wsj,What has Trump done as president so far? The W...,https://www.wsj.com/politics/policy/donald-tru...
23,27,2025-09-15-14-14-04 +0000,nypost,Charlie Kirk shooting is ‘the death of free sp...,https://nypost.com/2025/09/15/media/charlie-ki...
147,27,2025-09-15-04-43-10 +0000,nypost,Maniac NYC driver who mowed down teen girl aft...,https://nypost.com/2025/09/15/us-news/maniac-n...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
